# U-K formulation modal synthesis
## Usage example
### Author: piptouque


### Imports


In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from uk.data import GuitarBodyData, GuitarStringData, Excitation
from uk.structure import GuitarString, GuitarBody, ModalSimulation
from util import load_data_json, load_data_csv, make_modetime_dataframe

ImportError: cannot import name 'load_data_json' from 'util' (c:\Users\zliu7\Documents\M2 ATIAM\PAM\project codes\atiam_pam_hrparam\source\util\__init__.py)

### Load config

In [ ]:
string_data_path = pathlib.Path('./data/guitar_string_config.json')
body_data_path = pathlib.Path('./data/guitar_body_data.csv')
guitar_excitation_config = pathlib.Path('./data/guitar_excitation_config.json')
simulation_config = pathlib.Path('./data/simulation_config.json')
output_path = pathlib.Path('../out/uk')


string_data = load_data_json(GuitarStringData, string_data_path)
body_data = load_data_csv(GuitarBodyData, body_data_path)
f_ext_string = load_data_json(
    Excitation.make_triangular, guitar_excitation_config, l=string_data.l)
simulation = load_data_json(ModalSimulation, simulation_config)

print(f"String data: \n {string_data._param_dict}")
print(f"Body data: \n {body_data._param_dict}")
print(f"Simulation config \n {simulation._param_dict}")

x = np.linspace(0, string_data.l, 100)
f_x = [f_ext_string(u, 0) for u in x]

plt.plot(x, f_x)
plt.title("External force applied on the string")
plt.show()

### Solve system

In [ ]:
s = GuitarString(string_data)
b = GuitarBody(body_data)

# There is no external force applied to the body.
f_ext_body = Excitation.make_null()

# The string and body are initially at rest.
q_n_is = [np.zeros(n.shape, dtype=float) for i in range(2)]
dq_n_is = [np.zeros(n.shape, dtype=float) for i in range(2)]

# Run the simulation / solve the system.
t, q_ns, dq_ns, ddq_ns, ext_force_n_ts = simulation.run(
    [s, b], [f_ext_string, f_ext_body],
    q_n_is, dq_n_is)

# compute data frames from the result.
df_q_n = make_modetime_dataframe(q_ns[0], n, t)
df_dq_n = make_modetime_dataframe(dq_ns[0], n, t)
df_ddq_n = make_modetime_dataframe(ddq_ns[0], n, t)
df_ext_force_n_t = make_modetime_dataframe(ext_force_n_ts[0], n, t)

### Save output

In [ ]:
output_path.mkdir(parents=True, exist_ok=True)
df_q_n.to_csv(output_path / 'q_n.csv')
df_ddq_n.to_csv(output_path / 'dq_n.csv')
df_ddq_n.to_csv(output_path / 'ddq_n.csv')
df_ext_force_n_t.to_csv(output_path / 'ext_force_n_t.csv')

### Plot output?